In [1]:
import pandas as pd
import utility

pricesDataFolder = "prices_data/"
dfH4 = pd.read_csv(
    filepath_or_buffer=pricesDataFolder+"H4/[SP500]_H4_2014-03-20__2024-03-20.csv",
    delimiter="\t"
)

dfH4 = dfH4.rename(
    columns = {
        '<OPEN>':'open', 
        '<HIGH>':'high', 
        '<LOW>':'low', 
        '<CLOSE>':'close'
    }
)

dfH4['datetime'] = pd.to_datetime(dfH4['<DATE>'] + ' ' + dfH4['<TIME>'])
dfH4 = dfH4.drop(['<TICKVOL>', '<VOL>', '<SPREAD>', '<DATE>', '<TIME>'], axis=1)
dfH4 = utility.heikinashi(dfH4) #add heikin ashi candles
dfH4["shortTermMA"] = dfH4["close"].rolling(window=50).mean() # add moving average 50
dfH4["longTermMA"] = dfH4["close"].rolling(window=200).mean() # add moving average 200
dfH4 = utility.addHACandleColor(dfH4)

In [2]:

from actor_critic_training import actorCritictrainingLoop
from reinforcement_learning.trading_agent.actor_critic.agent import Agent
from strategies.Heikin_Ashi_Moving_Average_Strategy import HeikinAshiMovingAverage

agent = Agent()
strategy = HeikinAshiMovingAverage(agent, useSR="SRbyRL", useUpdateSl=True, uselongTermMA=False)

actorCritictrainingLoop(dfH4, strategy)

ModuleNotFoundError: No module named 'actor_critic'